Генетический метод

In [ ]:
import random
import copy
import tkinter as tk
from tkinter import ttk


POPULATION_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.1
SHIFT_DURATION = 9


PEAK_HOURS = [(7, 9), (17, 19)]


drivers = [
    {"name": "Водитель 1", "type": "A"},
    {"name": "Водитель 2", "type": "A"},
    {"name": "Водитель 3", "type": "B"},
    {"name": "Водитель 4", "type": "A"},
    {"name": "Водитель 5", "type": "B"},
    {"name": "Водитель 6", "type": "B"},
    {"name": "Водитель 7", "type": "A"},
    {"name": "Водитель 8", "type": "B"},
]

DAYS_OF_WEEK = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]


def is_peak(hour):
    for start, end in PEAK_HOURS:
        if start <= hour < end:
            return True
    return False


def create_driver_schedule(driver, start_hour):
    schedule = {}
    shift_start = start_hour
    shift_end = (shift_start + SHIFT_DURATION) % 24
    shift_hours = [(shift_start + i) % 24 for i in range(SHIFT_DURATION)]
    breaks = [shift_hours[4]]

    for hour in shift_hours:
        if hour == shift_start:
            schedule[hour] = 'Начало смены'
        elif hour in breaks:
            schedule[hour] = 'Перерыв'
        elif is_peak(hour):
            schedule[hour] = 'Поездка (час пик)'
        else:
            schedule[hour] = 'Поездка'

    schedule[shift_end] = 'Окончание смены'
    return schedule


def generate_random_day_schedule():
    day_schedule = []
    for driver in drivers:
        shift_start = random.randint(6, 18)
        driver_copy = copy.deepcopy(driver)
        driver_copy['start'] = shift_start
        day_schedule.append(driver_copy)
    return day_schedule


def fitness(day_schedule):
    penalty = 0
    peak_hours_coverage = 0

    for driver in day_schedule:
        shift = create_driver_schedule(driver, driver['start'])
        peak_hours_count = sum(1 for hour, task in shift.items() if is_peak(hour) and task == 'Поездка (час пик)')
        
        
        peak_hours_coverage += peak_hours_count * 2

        for hour, task in shift.items():
            if task == 'Перерыв' and is_peak(hour):
                penalty += 10

    return peak_hours_coverage - penalty



def crossover(parent1, parent2):
    child = []
    for d1, d2 in zip(parent1, parent2):
        if random.random() > 0.5:
            child.append(copy.deepcopy(d1))
        else:
            child.append(copy.deepcopy(d2))
    return child


def mutate(day_schedule):
    for driver in day_schedule:
        if random.random() < MUTATION_RATE:
            driver['start'] = random.randint(6, 18)
    return day_schedule


def genetic_algorithm():
    population = [generate_random_day_schedule() for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        population.sort(key=lambda s: -fitness(s))
        print(f"Generation {generation}, Best Fitness: {fitness(population[0])}")

        
        new_population = population[:10]

        
        while len(new_population) < POPULATION_SIZE:
            parent1, parent2 = random.sample(population[:20], 2)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)

        population = new_population

    best_schedule = population[0]
    return best_schedule


def gui(weekly_schedule):
    root = tk.Tk()
    root.title("Расписание водителей автобусов (Неделя)")
    root.geometry("1800x700")

    
    canvas = tk.Canvas(root)
    v_scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    h_scrollbar = ttk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=v_scrollbar.set, xscrollcommand=h_scrollbar.set)

    
    canvas.pack(side="left", fill="both", expand=True)
    v_scrollbar.pack(side="right", fill="y")
    h_scrollbar.pack(side="bottom", fill="x")

    
    def sort_hours(hours):
        return sorted(hours, key=lambda h: h if h >= 6 else h + 24)

    
    for day_idx, day in enumerate(DAYS_OF_WEEK):
        day_label = ttk.Label(scrollable_frame, text=f"{day}", font=("Arial", 14, "bold"))
        day_label.grid(row=day_idx * 2, column=0, padx=10, pady=10, sticky="w")

        day_schedule = weekly_schedule[day]
        for driver_idx, driver in enumerate(day_schedule):
            driver_schedule = create_driver_schedule(driver, driver['start'])
            driver_type = driver['type']
            frame = ttk.LabelFrame(scrollable_frame, text=f"{driver['name']} (Тип {driver_type})", padding=10)
            frame.grid(row=day_idx * 2 + 1, column=driver_idx, padx=10, pady=5, sticky="n")

            
            hours_sorted = sort_hours(driver_schedule.keys())
            for hour in hours_sorted:
                activity = driver_schedule[hour % 24]
                
                
                if 'Поездка' in activity:
                    route_number = random.randint(1, 2)
                    activity += f" (Маршрут №{route_number})"
                
                label = ttk.Label(frame, text=f"{hour % 24:02d}:00 - {activity}", anchor="w", padding=5)
                label.pack(fill="x")

    root.mainloop()


if __name__ == "__main__":
    weekly_schedule = {}
    for day in DAYS_OF_WEEK:
        print(f"Оптимизация для дня: {day}")
        best_day_schedule = genetic_algorithm()
        weekly_schedule[day] = best_day_schedule

    gui(weekly_schedule)


Оптимизация для дня: Понедельник
Generation 0, Best Fitness: 28
Generation 1, Best Fitness: 30
Generation 2, Best Fitness: 30
Generation 3, Best Fitness: 32
Generation 4, Best Fitness: 32
Generation 5, Best Fitness: 32
Generation 6, Best Fitness: 32
Generation 7, Best Fitness: 32
Generation 8, Best Fitness: 32
Generation 9, Best Fitness: 32
Generation 10, Best Fitness: 32
Generation 11, Best Fitness: 32
Generation 12, Best Fitness: 32
Generation 13, Best Fitness: 32
Generation 14, Best Fitness: 32
Generation 15, Best Fitness: 32
Generation 16, Best Fitness: 32
Generation 17, Best Fitness: 32
Generation 18, Best Fitness: 32
Generation 19, Best Fitness: 32
Generation 20, Best Fitness: 32
Generation 21, Best Fitness: 32
Generation 22, Best Fitness: 32
Generation 23, Best Fitness: 32
Generation 24, Best Fitness: 32
Generation 25, Best Fitness: 32
Generation 26, Best Fitness: 32
Generation 27, Best Fitness: 32
Generation 28, Best Fitness: 32
Generation 29, Best Fitness: 32
Generation 30, Be